# Sudoku solver

In [354]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [355]:
def cross(a, b):
    return [s + t for s in a for t in b]

## Setup boxes, units and peers

Let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:

- The individual squares at the intersection of rows and columns will be called __boxes__. These boxes will have labels 'A1', 'A2', ..., 'I9'.
- The complete rows, columns, and 3x3 squares, will be called __units__. Thus, each unit is a set of 9 boxes, and there are 27 units in total.
- For a particular box (such as 'A1'), its __peers__ will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square).

In [356]:
boxes = cross(rows, cols)

In [357]:
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') 
                              for cs in ('123','456','789')]

unitlist = row_units + column_units + square_units

In [358]:
#units_dict = dict((b, [u for u in unit_list if b in u]) for b in boxes)

In [359]:
def create_units_dict():
    units_dict = dict(zip(boxes, [[] for _ in boxes]))
    for unit in unitlist:
        for box in unit:
            units_dict[box].append(unit)

    return units_dict

units_dict = create_units_dict()

In [360]:
#peers_dict = dict((s, set(sum(units_dict[s], [])) - set([s])) for s in boxes)

In [361]:
def create_peers_dict():
    peers_dict = {}
    for box in boxes:
        blist = [b for u in units_dict[box] for b in u]
        box_peers = set(blist) - set([box])
        peers_dict[box] = box_peers
        
    return peers_dict

peers_dict = create_peers_dict()

In [362]:
def display(values):
    """Display the values as a 2-D grid.

    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [363]:
puzzle1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
puzzle2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'

In [364]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == len(boxes)
    
    values_string = '123456789'

    result = {}
    for i, v in enumerate(grid):
        if v == '.':
            result[boxes[i]] = values_string
        else:
            result[boxes[i]] = v
    
    return result
    

In [365]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_boxes = [box for box in values.keys() if len(values[box]) == 1]
    result = values.copy()
    for box in solved_boxes:
        for peer in peers_dict[box]:
            result[peer] = result[peer].replace(result[box], '')
            
    return result


In [366]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    values_string = '123456789'

    result = values.copy()
    for unit in unitlist:

        value_blist_dict = {v:[] for v in values_string}

        for box in unit:
            for v in values[box]:
                value_blist_dict[v].append(box)
        
        for v in value_blist_dict:
            blist = value_blist_dict[v]
            if len(blist) == 1:
                result[blist[0]] = v

    return result

In [367]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])

        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after

        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False

    return values

In [368]:
def search(values):
    """Using depth-first search and propagation, create a search tree and solve 
    the sudoku."""
    
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False
    if all(len(values[b]) == 1 for b in boxes): 
        return values
    
    # Choose one of the unfilled squares with the fewest possibilities
    bopts = []
    for b in boxes:
        if len(values[b]) > 1:
            bopts.append((len(values[b]), b))
    _, b = min(bopts)
#    _, b = min((len(values[b]), b) for b in boxes if len(values[b]) > 1)
    
    # Now use recursion to solve each one of the resulting sudokus, and if one 
    # returns a value (not False), return that answer!
    for v in values[b]:

        test_values = values.copy()
        test_values[b] = v

        test_result = search(test_values)
        if test_result:
            return test_result

In [369]:
values = search(grid_values(puzzle1))
display(values)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In [370]:
values = search(grid_values(puzzle2))
display(values)

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 
